# Group Project

As a Data Scientist, you are tasked to help these users find the most interesting articles
according to their preferred topics. You have a ***training dataset containing about 9500 news
articles, each assigned to one of the above topics***. In addition, (as in real life situation) the
dataset contains about ***48% of irrelevant articles*** (marked as IRRELEVANT) that do not
belong to any of the topics; hence the users are not interested in them. The distribution of
articles over topics is not uniform. There are some topics with large number of articles, and
some with very small number.

One day, 500 new articles have been published. This is your test set that has similar article
distribution over topics to the training set. ***Your task is to suggest up to 10 of the most relevant
articles from this set of 500 to each user***. The number of suggestions is limited to 10, because,
presumably, the users do not want to read more suggestions. It is possible, however, that some
topics within this test set have less than 10 articles. You also do not want to suggest 10 articles
if they are unlikely to be relevant, because you are concerned that the users may get
discouraged and stop using your application altogether. Therefore you need to take a balanced
approach, paying attention to not suggesting too many articles for rare topics.




### 0422 update: 

try considering stop word, but does not work

try using tdidf for word extraction, but shitty performance    

### Import Library

In [20]:
# Load libraries
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import collections


### Pre-processing

In [15]:
# Import text data
raw_training = pd.read_csv("training.csv")
raw_testing = pd.read_csv("test.csv")

# Create bag of words
count = CountVectorizer()
bag_of_words = count.fit_transform(raw_training["article_words"])

# Create feature matrix
X = bag_of_words

# Create bag of words
y = raw_training["topic"]

#######################Resampling Dataset#######################


# Reducing the effect imbalnced by deleting some irrelevant class
# "Irrelevant" classe has 4734 samples in the training data, try to reduce it into 2000
irrelevant = raw_training[raw_training["topic"] == "IRRELEVANT"]
remove_n = 2734
drop_indices = np.random.choice(irrelevant.index, remove_n, replace=False)
irrelevant = irrelevant.drop(drop_indices)

reduce_training =  pd.concat([raw_training[raw_training["topic"] != "IRRELEVANT"], irrelevant],ignore_index=True)
reduce_bag_of_words = count.fit_transform(reduce_training["article_words"])
R_X = reduce_bag_of_words
R_y = reduce_training["topic"]

# Icreasing the minor classes
# Increasing 
topic_class = raw_training[raw_training["topic"] != "IRRELEVANT"]
increase_training = pd.concat([topic_class, topic_class, topic_class, raw_training[raw_training["topic"] == "IRRELEVANT"]], ignore_index=True)
increase_bag_of_words = count.fit_transform(increase_training["article_words"])
I_X = increase_bag_of_words
I_y = increase_training["topic"]

# Considering stop words
count2 = CountVectorizer(stop_words='english')
sw_bag_of_words = count.fit_transform(increase_training["article_words"])
s_X = sw_bag_of_words
s_y = increase_training["topic"]


In [19]:
sw_bag_of_words

<19032x35822 sparse matrix of type '<class 'numpy.int64'>'
	with 1542963 stored elements in Compressed Sparse Row format>

In [3]:
y

0       FOREX MARKETS
1       MONEY MARKETS
2              SPORTS
3       FOREX MARKETS
4          IRRELEVANT
            ...      
9495          DEFENCE
9496       IRRELEVANT
9497    FOREX MARKETS
9498       IRRELEVANT
9499    FOREX MARKETS
Name: topic, Length: 9500, dtype: object

In [4]:
#print('Features:' , count.get_feature_names())# 檢視feature names
#print('Values: \n', X.toarray())

### Defining Method
Defining different model function to be used later in the k fold validation


In [5]:
# Using specific model "method", return specifc score "score" by cross validation

def Model_Score (X, y, method, k):
    clf = method
    
    accuracy_scores = cross_val_score(clf, X, y, cv=k, scoring="accuracy")
    precision_scores = cross_val_score(clf, X, y, cv=k, scoring="precision_macro")
    recall_scores = cross_val_score(clf, X, y, cv=k, scoring="recall_macro")
    f1_scores = cross_val_score(clf, X, y, cv=k, scoring="f1_macro")
    
    return np.mean(accuracy_scores), np.mean(precision_scores), np.mean(recall_scores), np.mean(f1_scores)

#def Model_report ()
    


### Training Model (Naive Bayes)
Using K-fold validation to split the training data and validation data. Use the average score of the validation sets to evaluate the performance of the model

In [16]:
# creating 10 fold for k-fold validation
#A = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [5, 6], [7, 2], [5, 4], [7, 5], [10, 4]]) 
b = np.array([1, 2, 3, 4,5,6,7,8,9,10])

# k-fold split number 10 
k = 10

# without doing any data cleaning

bernoulliNB_accuracy, bernoulliNB_precision, bernoulliNB_recall, bernoulliNB_f1 = Model_Score(X, y, BernoulliNB(), 10)
multinomialNB_accuracy, multinomialNB_precision, multinomialNB_recall, multinomialNB_f1 = Model_Score(X, y, MultinomialNB(), 10)


# multinomialNB using uniformed distribution
multiNB_accuracy2, multiNB_precision2, multiNB_recall2, multiNB_f12 = Model_Score(X, y, MultinomialNB(fit_prior = False), 10)


# reduce irrelevant samples
R_accuracy, R_precision, R_recall, R_f1 = Model_Score(R_X, R_y, MultinomialNB(), 10)


# Icreasing the minor classes
I_accuracy, I_precision, I_recall, I_f1 = Model_Score(I_X, I_y, MultinomialNB(), 10)


# Try considering stop words
s_accuracy, s_precision, s_recall, s_f1 = Model_Score(s_X, s_y, MultinomialNB(), 10)







# Try Classification report
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

clf_BernoulliNB = BernoulliNB()
model_BernoulliNB = clf_BernoulliNB.fit(X_train, y_train)

clf_MultinomialNB = MultinomialNB()
model_MultinomialNB = clf_MultinomialNB.fit(X_train, y_train)


predicted_BernoulliNB = model_BernoulliNB.predict(X_valid)
predicted_MultinomialNB = model_MultinomialNB.predict(X_valid)

I_X_train, I_X_valid, I_y_train, I_y_valid = train_test_split(I_X, I_y, test_size=0.20, random_state=42)
model_MultinomialNB_2 = clf_MultinomialNB.fit(I_X_train, I_y_train)
predicted_MultinomialNB_2 = model_MultinomialNB_2.predict(I_X_valid)





# still need to deal with 


# 1. irrelevant articles

# 2. The distribution of topics are not uniform

# 3. select the features

# 4. maybe can give penalty to the misclassifying




# multinomialNB_accuracy = Model_Score(X, y, MultinomialNB(), 10, "accuracy")


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\m

In [17]:
print("====================================================================================")
print("Without doing any data cleaning, the score of bernoulliNB,\naccuracy:  " + str(bernoulliNB_accuracy) +
     "\nprecision: " + str(bernoulliNB_precision) + "\nrecall:    " + str(bernoulliNB_recall) + "\nf1:        " +
     str(bernoulliNB_f1))
#print("\nClassification Report for bernoulliNB:\n")
#print(classification_report(y_valid, predicted_BernoulliNB))

print("====================================================================================")

print("Without doing any data cleaning, the score of multinomialNB,\naccuracy:  " + str(multinomialNB_accuracy) +
     "\nprecision: " + str(multinomialNB_precision) + "\nrecall:    " + str(multinomialNB_recall) + "\nf1:        " +
     str(multinomialNB_f1))
#print("\nClassification Report for multinomialNB:\n")
#print(classification_report(y_valid, predicted_MultinomialNB))

print("====================================================================================")

print("Setting uniformed prior, the score of multinomialNB,\naccuracy:  " + str(multiNB_accuracy2) +
     "\nprecision: " + str(multiNB_precision2) + "\nrecall:    " + str(multiNB_recall2) + "\nf1:        " +
     str(multiNB_f12))
#print("\nClassification Report for multinomialNB:\n")
#print(classification_report(y_valid, predicted_MultinomialNB))

print("====================================================================================")

print("Reduce the case of irrelevant, the score of multinomialNB,\naccuracy:  " + str(R_accuracy) +
     "\nprecision: " + str(R_precision) + "\nrecall:    " + str(R_recall) + "\nf1:        " +
     str(R_f1))

print("====================================================================================")

print("Increase(Copy) the case of topic classes, the score of multinomialNB,\naccuracy:  " + str(I_accuracy) +
     "\nprecision: " + str(I_precision) + "\nrecall:    " + str(I_recall) + "\nf1:        " +
     str(I_f1))

print("====================================================================================")

print("Considering about stop words, the score of multinomialNB,\naccuracy:  " + str(s_accuracy) +
     "\nprecision: " + str(s_precision) + "\nrecall:    " + str(s_recall) + "\nf1:        " +
     str(s_f1))

print("\nClassification Report for multinomialNB after Resampling Dataset:\n")
print(classification_report(I_y_valid, predicted_MultinomialNB_2))



Without doing any data cleaning, the score of bernoulliNB,
accuracy:  0.7038942167520311
precision: 0.37212369276862634
recall:    0.2819154032170111
f1:        0.28388746010825894
Without doing any data cleaning, the score of multinomialNB,
accuracy:  0.7358161832731782
precision: 0.6306736067435463
recall:    0.5526188603489018
f1:        0.5583352913640214
Setting uniformed prior, the score of multinomialNB,
accuracy:  0.7338169401702015
precision: 0.6325643464367477
recall:    0.5709569773293275
f1:        0.5712769594923721
Reduce the case of irrelevant, the score of multinomialNB,
accuracy:  0.7143252813914881
precision: 0.7186671782041083
recall:    0.6039261812740762
f1:        0.6259661645274954
Increase(Copy) the case of topic classes, the score of multinomialNB,
accuracy:  0.8046992057147376
precision: 0.8135680332176245
recall:    0.8751822879751108
f1:        0.8346417211219871
Considering about stop words, the score of multinomialNB,
accuracy:  0.8046992057147376
precisio

### Final Test Try

In [8]:
bag_of_words_test = count.transform(raw_testing["article_words"])

# Create feature matrix
X_test = bag_of_words_test
y_test = raw_testing["topic"]
predicted_testcase = model_MultinomialNB_2.predict(X_test)
print("test case result(using the best performanced multinomailNB2):")
print("acccuracy: " + str(accuracy_score(y_test, predicted_testcase)))
print("precision: " + str(precision_score(y_test, predicted_testcase, average='macro')))
print("recall: " + str(recall_score(y_test, predicted_testcase, average='macro')))
print("f1: " + str(f1_score(y_test, predicted_testcase, average='macro')))
print(classification_report(y_test, predicted_testcase))


test case result(using the best performanced multinomailNB2):
acccuracy: 0.706
precision: 0.6103673162341499
recall: 0.6969960912466633
f1: 0.6072519053853263
                                  precision    recall  f1-score   support

      ARTS CULTURE ENTERTAINMENT       0.33      0.67      0.44         3
BIOGRAPHIES PERSONALITIES PEOPLE       0.73      0.53      0.62        15
                         DEFENCE       0.44      0.92      0.60        13
                DOMESTIC MARKETS       0.17      0.50      0.25         2
                   FOREX MARKETS       0.38      0.52      0.44        48
                          HEALTH       0.68      0.93      0.79        14
                      IRRELEVANT       0.94      0.65      0.77       266
                   MONEY MARKETS       0.49      0.75      0.59        69
          SCIENCE AND TECHNOLOGY       1.00      0.33      0.50         3
                  SHARE LISTINGS       0.60      0.86      0.71         7
                          

### Using TF-IDF for word extraction

In [28]:
vectorizer = TfidfVectorizer()

#tf-idf 
tfidf = vectorizer.fit_transform(raw_training["article_words"])
tfidf0_X = tfidf
tfidf0_y = raw_training["topic"]
# score using tf-idf words extraction
tfidf0_accuracy, tfidf0_precision, tfidf0_recall, tfidf0_f1 = Model_Score(tfidf0_X, tfidf0_y, MultinomialNB(), 10)


#tf-idf with increased data
increase_tfidf = vectorizer.fit_transform(increase_training["article_words"])
tfidf_X = increase_tfidf
tfidf_y = increase_training["topic"]

# score using tf-idf words extraction
tfidf_accuracy, tfidf_precision, tfidf_recall, tfidf_f1 = Model_Score(tfidf_X, tfidf_y, MultinomialNB(), 10)



print("====================================================================================")

print("Using tf-idf for words extraction, the score of multinomialNB,\naccuracy:  " + str(tfidf0_accuracy) +
     "\nprecision: " + str(tfidf0_precision) + "\nrecall:    " + str(tfidf0_recall) + "\nf1:        " +
     str(tfidf0_f1))
# tf-idf with increased data





print("====================================================================================")

print("Using tf-idf for words extraction(with increased data), the score of multinomialNB,\naccuracy:  " + str(tfidf_accuracy) +
     "\nprecision: " + str(tfidf_precision) + "\nrecall:    " + str(tfidf_recall) + "\nf1:        " +
     str(tfidf_f1))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\m

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Using tf-idf for words extraction, the score of multinomialNB,
accuracy:  0.6560008656647193
precision: 0.200763162792237
recall:    0.20202644451201693
f1:        0.1962810796307532
Using tf-idf for words extraction(with increased data), the score of multinomialNB,
accuracy:  0.6519025860434599
precision: 0.6564512514734997
recall:    0.28690327479646266
f1:        0.27558590791252524


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Training Model (Decision Tree)
Using K-fold validation to split the training data and validation data. Use the average score of the validation sets to evaluate the performance of the model


In [11]:
collections.Counter(y)

Counter({'FOREX MARKETS': 845,
         'MONEY MARKETS': 1673,
         'SPORTS': 1102,
         'IRRELEVANT': 4734,
         'SHARE LISTINGS': 218,
         'BIOGRAPHIES PERSONALITIES PEOPLE': 167,
         'DOMESTIC MARKETS': 133,
         'DEFENCE': 258,
         'SCIENCE AND TECHNOLOGY': 70,
         'HEALTH': 183,
         'ARTS CULTURE ENTERTAINMENT': 117})

In [14]:
count.get_feature_names()

['_____',
 '______',
 '_______',
 '____________',
 '_bret',
 'a1',
 'a1b1',
 'a2',
 'a3',
 'a300',
 'a300b4',
 'a320',
 'a330',
 'a340',
 'a4',
 'a5',
 'a6',
 'a7',
 'a78',
 'a7e',
 'aa',
 'aa1',
 'aa2',
 'aa3',
 'aaa',
 'aacount',
 'aad',
 'aadj',
 'aag',
 'aah',
 'aahp',
 'aair',
 'aalborg',
 'aalst',
 'aalton',
 'aaltonaa',
 'aama',
 'aamir',
 'aamodt',
 'aandewiel',
 'aap',
 'aapc',
 'aapt',
 'aaqib',
 'aaquib',
 'aardolie',
 'aardvark',
 'aargau',
 'aarhus',
 'aaron',
 'aarp',
 'aarts',
 'aasgard',
 'aash',
 'aato',
 'ab',
 'aba',
 'abab',
 'ababia',
 'abac',
 'abach',
 'aback',
 'abacus',
 'abad',
 'abandon',
 'abar',
 'abat',
 'abattoir',
 'abax',
 'abay',
 'abb',
 'abba',
 'abbasbeyl',
 'abbasov',
 'abbey',
 'abbot',
 'abbrev',
 'abc',
 'abc1',
 'abcd',
 'abciximab',
 'abcl',
 'abcp',
 'abdal',
 'abdel',
 'abdelkad',
 'abdelwahhab',
 'abderdeen',
 'abderrahim',
 'abderrahm',
 'abdicat',
 'abdom',
 'abdomin',
 'abdoujaparov',
 'abduc',
 'abdul',
 'abdulah',
 'abdull',
 'abdullah